# Day 9

## part 1

- digits alternate representing:
    - files
    - free space
- File ID starts at 0 and increments each block
- to compress the files - move them one at a time from the lend of the disk into free space at the beginning.
- the checksum is the sum of the products of each file id and block index

In [3]:
from dataclasses import dataclass
import logging

from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig as PC, markdown
)

log = logging.getLogger('day 9')
logging.basicConfig(level=logging.INFO)

In [ ]:
# lets work on the example first
example = '2333133121414131402'
ex_disk = [int(char) for char in example]

@dataclass
class FileBlocks:
    id: int
    start: int
    end: int

    @property
    def len(self) -> int:
        return self.end - self.start

    def range(self):
        return range(self.start, self.end)

@dataclass
class Disk:
    files: list[FileBlocks]

    @property
    def len(self) -> int:
        return len(self.files)
    
    @property
    def debug_str(self) -> str:
        """returns a string like the example for debugging"""
        output = ['.']*self.files[-1].end
        previous = None
        for fileblock in self.files:
            for index in fileblock.range():
                output[index] = f'{fileblock.id}'
        return ''.join(output)


def defrag(input_disk: list[int]) -> Disk:
    """returns the defragmented disk"""
    # first convert to a Disk type object
    disk = Disk([])
    is_file = True
    start = 0
    file_id = 0
    for digit in input_disk:
        if is_file:
            new_block = FileBlocks(file_id, start, start + digit)
            disk.files.append(new_block)
            log.debug(f'added {new_block=}')
            file_id += 1
        else:
            log.debug(f'Incremented start by {digit}')
        start += digit
        is_file = not is_file
    log.info(f'Mapped disk with {disk.len=}')

    return disk

log.setLevel(logging.DEBUG)
ex_defrag = defrag(ex_disk)
log.info(f'{ex_defrag.debug_str}')

DEBUG:day 9:added new_block=FileBlocks(id=0, start=0, end=2)
DEBUG:day 9:Incremented start by 3
DEBUG:day 9:added new_block=FileBlocks(id=1, start=5, end=8)
DEBUG:day 9:Incremented start by 3
DEBUG:day 9:added new_block=FileBlocks(id=2, start=11, end=12)
DEBUG:day 9:Incremented start by 3
DEBUG:day 9:added new_block=FileBlocks(id=3, start=15, end=18)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=FileBlocks(id=4, start=19, end=21)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=FileBlocks(id=5, start=22, end=26)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=FileBlocks(id=6, start=27, end=31)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=FileBlocks(id=7, start=32, end=35)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=FileBlocks(id=8, start=36, end=40)
DEBUG:day 9:Incremented start by 0
DEBUG:day 9:added new_block=FileBlocks(id=9, start=40, end=42)
INFO:day 9:Mapped disk with disk.len=10
INFO:day 9:00...111

In [ ]:
parser = PC('', int)
disk = parse_from_file('day_9.txt', parser)